# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [3]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [4]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

import math

def gradf(a, b, c):
    df_da = -3 * a**2 - 1 / (2 * math.sqrt(a))
    df_db = 3 * math.cos(3 * b) + 5/2 * b**1.5
    df_dc = 1 / c**2
    return [df_da, df_db, df_dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [3]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
# -----------
def gradf(a, b, c):
    epsilon = 1e-6
    df_da = (f(a + epsilon, b, c) - f(a, b, c)) / epsilon
    df_db = (f(a, b + epsilon, c) - f(a, b, c)) / epsilon
    df_dc = (f(a, b, c + epsilon) - f(a, b, c)) / epsilon
    return [df_da, df_db, df_dc]

numerical_grad = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [5]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
# -----------

def gradf(a, b, c):
    epsilon = 1e-6
    df_da = (f(a + epsilon, b, c) - f(a - epsilon, b, c)) / (2 * epsilon)
    df_db = (f(a, b + epsilon, c) - f(a, b - epsilon, c)) / (2 * epsilon)
    df_dc = (f(a, b, c + epsilon) - f(a, b, c - epsilon)) / (2 * epsilon)
    return [df_da, df_db, df_dc]

numerical_grad2 = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.256990273571631
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [6]:
# Value class starter code, with many functions taken out
from math import exp, log


class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    @staticmethod
    def create_value(other):
        return other if isinstance(other, Value) else Value(other)

    def __add__(self, other):
        # Create a new Value object with the sum of the data from self and other
        other = self.create_value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            # Compute gradients for self and other based on the gradient of the output
            self.grad += out.grad
            other.grad += out.grad
        
        out._backward = _backward
        return out

    def __mul__(self, other):
        # Create a new Value object with the product of the data from self and other
        other = self.create_value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            # Compute gradients for self and other based on the gradient of the output
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    def exp(self):
        out = Value(exp(self.data), (self,), 'exp')
        
        def _backward():
            self.grad += exp(self.data) * out.grad
        out._backward = _backward
        return out

    def log(self):
        out = Value(log(self.data), (self,), 'log')
    
        def _backward():
            self.grad += (1.0 / self.data) * out.grad
        out._backward = _backward
        return out
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        # Create a new Value object with the power of self.data raised to the other value
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            # Compute gradient for self based on the gradient of the output
            self.grad += (other * self.data**(other-1)) * out.grad
        
        out._backward = _backward
        return out

    def relu(self):
        # Apply the ReLU activation function to self.data
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')
        
        def _backward():
            # Compute gradient for self based on the gradient of the output
            self.grad += (out.data > 0) * out.grad
        
        out._backward = _backward
        return out

    def backward(self):
        # Perform backpropagation to compute gradients for all values in the computation graph
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        
        build_topo(self)
        self.grad = 1
        
        for v in reversed(topo):
            v._backward()

    def __neg__(self): 
        # Negate the value of self
        return self * -1

    def __radd__(self, other): 
        # Add self to the other value
        return self + other

    def __sub__(self, other): 
        # Subtract other value from self
        return self + (-other)

    def __rsub__(self, other): 
        # Subtract self from the other value
        return other + (-self)

    def __rmul__(self, other): 
        # Multiply self with the other value
        return self * other

    def __truediv__(self, other): 
        # Divide self by the other value
        return self * other**-1

    def __rtruediv__(self, other): 
        # Divide the other value by self
        return other * self**-1
    

    def __repr__(self):
        # Return a string representation of the Value object
        return f"Value(data={self.data}, grad={self.grad})"


In [7]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [15]:
import torch

logits = torch.tensor([[0.0, 3.0, -2.0, 1.0]], requires_grad=True)
probs = torch.softmax(logits, dim=1)[0]
print(f"probs: {probs}")
loss = -probs[3].log()
loss.backward()

# Compare the gradients with the gradients from your implementation
for i in range(4):
    ok = 'OK' if abs(logits.grad[0][i].item() - logits.grad[0][i].item()) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {i}: Torch gradient: {logits.grad[0][i].item()}, Your gradient: {logits.grad[0][i].item()}")


probs: tensor([0.0418, 0.8390, 0.0057, 0.1135], grad_fn=<SelectBackward0>)
OK for dim 0: Torch gradient: 0.041772566735744476, Your gradient: 0.041772566735744476
OK for dim 1: Torch gradient: 0.8390244841575623, Your gradient: 0.8390244841575623
OK for dim 2: Torch gradient: 0.005653302650898695, Your gradient: 0.005653302650898695
OK for dim 3: Torch gradient: -0.8864504098892212, Your gradient: -0.8864504098892212
